In [1]:
import pickle
import numpy as np
from itertools import chain, combinations

In [2]:
PIK = "../../data/Syndata7C.dat"

with open(PIK, "rb") as f:
    trueW = pickle.load(f)
    trueA = pickle.load(f)
    trueX = pickle.load(f)
    trueZ = pickle.load(f)
    trueRho = pickle.load(f)
    truePhi = pickle.load(f)
    trueGam = pickle.load(f)
    Nt = pickle.load(f)
    eta1 = pickle.load(f)
    eta2 = pickle.load(f)
    alpha = pickle.load(f)
    beta = pickle.load(f)
    

In [3]:
D, T = trueW.shape[1], trueW.shape[2]

In [4]:
def init_X(K, T):
    return np.repeat(1/2, K*T).reshape(K,T)

def init_A(W, Nt, K):
    """
        Return A for the initilization
        We first assign active cluster for a patient, then arrange cluster assignment based on it
    """
    maxNt, D, T = W.shape
    initA = np.zeros((maxNt, D, T), dtype=np.int32) - 1
    
    for t in range(T):
        for i in range(Nt[t]):
            if sum(W[i,:,t]) > 0:
                choice = list(chain(*(combinations(np.arange(K), x) for x in range(1, K+1))))
                cls = np.random.choice(choice, size=1)
                initA[i, :, t] = np.random.choice(np.array(cls[0]), size=D)  
    return initA

def init_Gam():
    return np.array([trueGam])

def init_Z(K, T, Nt, A):
    """
        Returns intiailization of Z where Z_itk = 1 for N_itk > 0 from initA
    """
    maxNt = np.max(Nt)
    initZ = np.zeros((maxNt, K, T), dtype=np.int32) - 1
    
    for t in range(T):
        for i in range(Nt[t]):
            temp = np.repeat(0,K)
            if A[i,0,t] > -1:
                temp[np.unique(A[i,:,t])] = np.repeat(1, np.unique(A[i,:,t]).shape[0])
            initZ[i, :, t] = temp # assign 0 or 1 randomly
    return initZ

In [5]:
D, T = trueW.shape[1], trueW.shape[2]
K = 3
print("Setting of the experiment\n")
print("Number of observations across time : {}\n".format(Nt))
print("Number of underlying clusters K: {}\n".format(K))
print("Number of diseases D: {}\n".format(D))
print("Number of time points T: {}\n".format(T))

Setting of the experiment

Number of observations across time : [150 150 150 150 150 150]

Number of underlying clusters K: 3

Number of diseases D: 20

Number of time points T: 6



In [9]:
np.random.seed(13)

initA = init_A(trueW, Nt, K)
initZ = init_Z(K, T, Nt, initA)
initX = init_X(K, T)
initPhi = np.repeat(3.0, K)

/var/folders/kv/8zn4jf1s0jxch5rjz712r5t00000gn/T/ipykernel_97617/2601361976.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cls = np.random.choice(choice, size=1)


In [10]:
with open('init.dat', "wb") as f:
    pickle.dump(initA, f)
    pickle.dump(initZ, f)
    pickle.dump(initX, f)
    pickle.dump(initPhi, f)